# METAR Data Analysis Project

METAR is the acronym for Meteorological Aerodrome Report. It is internationally recognized shorthand for weather data used by the aviation community.

Different abbreviations and codes from US Government National Weather Service: [source](https://www.weather.gov/media/wrh/mesowest/metar_decode_key.pdf)

### The Problem:

A Flight school is looking to expand to new locations. You're given METAR data to identify the 10 best and 10 worst locations based on the following criteria:

* Visibility of 10 statute miles or greater 

* Cloud ceiling of 3,000 ft above ground or higher

* Winds less than 15 kts

----

[Airport Data Link](https://data.humdata.org/dataset/ourairports-usa?)

[Kaggle Airport Data](https://www.kaggle.com/datasets/aravindram11/list-of-us-airports)

[US Cities Population](https://catalog.data.gov/dataset/500-cities-city-level-data-gis-friendly-format-2019-release)

### Imports and Reading in Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
import json
import utils
import warnings

# Data File Paths
# metar = '../data/metar_export.txt'
# us_geojson = '../data/gz_2010_us_040_00_500k.json'
# airports = '../data/airports.csv'
# population = './data.500_Cities__City-level_Data__GIS_Friendly_Format___2019_release.csv'

# Column Names Hardcoded
col_names = ['date', 'id', 'obsv_time', 'automated', 'wind_speed', 'visibility', 'cloud_cover', \
            'temp_dewpt', 'altimeter', 'altimeter_remarks', 'ao2']

df = pd.read_csv('../data/metar_export.txt', delimiter=' ', header = None, names = col_names)
airports = pd.read_csv('../data/airports.csv')
us_pop = pd.read_csv('../data/500_Cities__City-level_Data__GIS_Friendly_Format___2019_release.csv')

print(df.head())
print('='*40)
print()
df.info()

         date              id obsv_time automated wind_speed visibility  \
0  2015-03-25  21:15:00\tKCXP   252115Z      AUTO    08005KT       10SM   
1  2015-03-25  21:13:00\tKWRB   252113Z      AUTO    06005KT       10SM   
2  2015-03-25  21:12:00\tKTUL   252112Z   14006KT       10SM         TS   
3  2015-03-25  21:11:00\tKDRT   252111Z      AUTO    13007KT       10SM   
4  2015-03-25  21:10:00\tKBRL   252110Z      AUTO    33009KT        8SM   

  cloud_cover temp_dewpt altimeter altimeter_remarks    ao2  
0         CLR     17/M01     A3033               RMK    AO2  
1      BKN014     OVC024     20/17             A3009    RMK  
2      BKN035     BKN120    BKN250             24/16  A2975  
3      SCT023      22/17     A2986               RMK    AO2  
4      SCT016      07/03     A3006               RMK    AO2  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7731241 entries, 0 to 7731240
Data columns (total 11 columns):
 #   Column             Dtype 
---  ------             ----- 
 

--- 
Need to convert date, windspeed, and visibility to numeric data types.

As with many of the dataframes in this project, not all columns will be used.

---

In [2]:
# Checking population Data

print(us_pop.head())
print('='*40)
us_pop.info()

  StateAbbr   PlaceName  PlaceFIPS  Population2010  ACCESS2_CrudePrev  \
0        AL  Birmingham     107000          212237               20.3   
1        AL      Hoover     135896           81619               11.5   
2        AL  Huntsville     137000          180105               16.4   
3        AL      Mobile     150000          195111               18.1   
4        AL  Montgomery     151000          205764               19.0   

  ACCESS2_Crude95CI  ACCESS2_AdjPrev ACCESS2_Adj95CI  ARTHRITIS_CrudePrev  \
0      (19.9, 20.6)             20.6    (20.2, 21.0)                 29.8   
1      (11.0, 12.0)             11.8    (11.3, 12.3)                 25.3   
2      (16.0, 16.9)             16.6    (16.1, 17.0)                 28.9   
3      (17.8, 18.5)             18.5    (18.1, 18.9)                 30.2   
4      (18.5, 19.5)             19.1    (18.7, 19.6)                 28.3   

  ARTHRITIS_Crude95CI  ...  SLEEP_Adj95CI STROKE_CrudePrev  STROKE_Crude95CI  \
0        (29.6, 29

In [3]:
# Push to EDA folder
# Filtering only columns of interest
us_pop = us_pop.loc[:, ['StateAbbr', 'PlaceName', 'Population2010', 'Geolocation']].copy()
# Seeing top 20 Cities
us_pop.sort_values(by = 'Population2010', ascending = False).head(20)

,StateAbbr,PlaceName,Population2010,Geolocation
342,NY,New York,8175133,"(40.69496068900, -73.9313850409)"
74,CA,Los Angeles,3792621,"(34.11822778980, -118.408500088)"
222,IL,Chicago,2695598,"(41.83729506150, -87.6862308732)"
429,TX,Houston,2099451,"(29.78066913960, -95.3860033966)"
388,PA,Philadelphia,1526006,"(40.00931478080, -75.1333888571)"
13,AZ,Phoenix,1445632,"(33.57241386950, -112.088995222)"
451,TX,San Antonio,1327407,"(29.47214753330, -98.5246763525)"
111,CA,San Diego,1307402,"(32.83556394180, -117.119792061)"
421,TX,Dallas,1197816,"(32.79398040660, -96.7656929463)"
213,HI,Honolulu,953207,"(21.45880393050, -157.973296737)"


In [4]:
# cleaning up latitude and longitude in us_pop dataframe.
us_pop.head()

,StateAbbr,PlaceName,Population2010,Geolocation
0,AL,Birmingham,212237,"(33.52756637730, -86.7988174678)"
1,AL,Hoover,81619,"(33.37676027290, -86.8051937568)"
2,AL,Huntsville,180105,"(34.69896926710, -86.6387042882)"
3,AL,Mobile,195111,"(30.67762486480, -88.1184482714)"
4,AL,Montgomery,205764,"(32.34726453330, -86.2677059552)"


In [5]:
# Testing split for Geolocation column
us_pop['Geolocation'].str.split(pat = ',', expand = True)

,0,1
0,(33.52756637730,-86.7988174678)
1,(33.37676027290,-86.8051937568)
2,(34.69896926710,-86.6387042882)
3,(30.67762486480,-88.1184482714)
4,(32.34726453330,-86.2677059552)
...,...,...
495,(43.08098656940,-89.3915106344)
496,(43.06412589250,-87.9672412429)
497,(42.72745994940,-87.8134530240)
498,(43.00933322150,-88.2457679157)


In [6]:
us_pop = pd.merge(us_pop, us_pop['Geolocation'].str.split(pat = ',', expand = True), left_index = True, right_index = True)

us_pop = us_pop.iloc[:, :6].copy()

us_pop.head()

,StateAbbr,PlaceName,Population2010,Geolocation,0,1
0,AL,Birmingham,212237,"(33.52756637730, -86.7988174678)",(33.52756637730,-86.7988174678)
1,AL,Hoover,81619,"(33.37676027290, -86.8051937568)",(33.37676027290,-86.8051937568)
2,AL,Huntsville,180105,"(34.69896926710, -86.6387042882)",(34.69896926710,-86.6387042882)
3,AL,Mobile,195111,"(30.67762486480, -88.1184482714)",(30.67762486480,-88.1184482714)
4,AL,Montgomery,205764,"(32.34726453330, -86.2677059552)",(32.34726453330,-86.2677059552)


In [7]:
# Naming coordinates columns UPPER CASE for merging with METAR data
us_pop.columns = ['state', 'city', 'population2010', 'geolocation', 'latitude',
       'longitude']

us_pop.head()

,state,city,population2010,geolocation,latitude,longitude
0,AL,Birmingham,212237,"(33.52756637730, -86.7988174678)",(33.52756637730,-86.7988174678)
1,AL,Hoover,81619,"(33.37676027290, -86.8051937568)",(33.37676027290,-86.8051937568)
2,AL,Huntsville,180105,"(34.69896926710, -86.6387042882)",(34.69896926710,-86.6387042882)
3,AL,Mobile,195111,"(30.67762486480, -88.1184482714)",(30.67762486480,-88.1184482714)
4,AL,Montgomery,205764,"(32.34726453330, -86.2677059552)",(32.34726453330,-86.2677059552)


In [8]:
# Removing unnecessary characters in coordinate columns to convert to floats
us_pop['latitude'] = us_pop['latitude'].apply(lambda x: x.replace('(', '')).astype(float)
us_pop['longitude'] = us_pop['longitude'].apply(lambda x: x.replace(')', '')).astype(float)

us_pop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   state           500 non-null    object 
 1   city            500 non-null    object 
 2   population2010  500 non-null    int64  
 3   geolocation     500 non-null    object 
 4   latitude        500 non-null    float64
 5   longitude       500 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 23.6+ KB


In [9]:
# checking first few rows to see if any changes need to be made
# Dropping Geolocation since coordinates columns are in float format
us_pop.drop(columns = 'geolocation', inplace = True)

us_pop.head()

,state,city,population2010,latitude,longitude
0,AL,Birmingham,212237,33.527566,-86.798817
1,AL,Hoover,81619,33.376760,-86.805194
2,AL,Huntsville,180105,34.698969,-86.638704
3,AL,Mobile,195111,30.677625,-88.118448
4,AL,Montgomery,205764,32.347265,-86.267706


In [10]:
# Population looks good, exporting as a .csv
us_pop.to_csv('../data/us_pop.csv', index = False)

---

## Looking at .head() and .info() of airports dataset.

In [12]:
print(airports.head())
print('='*40) # line break
airports.info()

  iata                              airport           city state country  \
0  ABQ            Albuquerque International    Albuquerque    NM     USA   
1  ANC  Ted Stevens Anchorage International      Anchorage    AK     USA   
2  ATL    William B Hartsfield-Atlanta Intl        Atlanta    GA     USA   
3  AUS       Austin-Bergstrom International         Austin    TX     USA   
4  BDL                Bradley International  Windsor Locks    CT     USA   

    latitude   longitude  
0  35.040222 -106.609194  
1  61.174320 -149.996186  
2  33.640444  -84.426944  
3  30.194533  -97.669872  
4  41.938874  -72.683228  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   iata       341 non-null    object 
 1   airport    341 non-null    object 
 2   city       337 non-null    object 
 3   state      337 non-null    object 
 4   country    341 non-null    objec

In [13]:
print(f"Number of unique airports: {airports['iata'].nunique()}")
airports['iata'].unique()

Number of unique airports: 341


array(['ABQ', 'ANC', 'ATL', 'AUS', 'BDL', 'BHM', 'BNA', 'BOS', 'BUF',
       'BUR', 'BWI', 'CHS', 'CLE', 'CLT', 'CMH', 'CVG', 'DAL', 'DCA',
       'DEN', 'DFW', 'DTW', 'ELP', 'EWR', 'FLL', 'HNL', 'HOU', 'IAD',
       'IAH', 'IND', 'JAX', 'JFK', 'LAS', 'LAX', 'LGA', 'LIT', 'MCI',
       'MCO', 'MDW', 'MEM', 'MIA', 'MKE', 'MSP', 'MSY', 'OAK', 'OGG',
       'OKC', 'OMA', 'ONT', 'ORD', 'ORF', 'PBI', 'PDX', 'PHL', 'PHX',
       'PIT', 'PVD', 'RDU', 'RIC', 'RNO', 'RSW', 'SAN', 'SAT', 'SDF',
       'SEA', 'SFO', 'SJC', 'SJU', 'SLC', 'SMF', 'SNA', 'STL', 'TPA',
       'TUL', 'TUS', 'ABE', 'ABI', 'ABR', 'ABY', 'ACK', 'ACT', 'ACV',
       'ACY', 'ADK', 'ADQ', 'AEX', 'AGS', 'AKN', 'ALB', 'ALO', 'AMA',
       'APN', 'ART', 'ASE', 'ATW', 'AVL', 'AVP', 'AZO', 'BET', 'BFL',
       'BGM', 'BGR', 'BIL', 'BIS', 'BJI', 'BLI', 'BMI', 'BOI', 'BPT',
       'BQK', 'BQN', 'BRD', 'BRO', 'BRW', 'BTM', 'BTR', 'BTV', 'BZN',
       'CAE', 'CAK', 'CDC', 'CDV', 'CEC', 'CHA', 'CHO', 'CIC', 'CID',
       'CIU', 'CLD',

In [14]:
# already lower-cased columsn in previous iteration
# airports.columns = airports.columns.str.lower()

airports.head()

,iata,airport,city,state,country,latitude,longitude
0,ABQ,Albuquerque International,Albuquerque,NM,USA,35.040222,-106.609194
1,ANC,Ted Stevens Anchorage International,Anchorage,AK,USA,61.174320,-149.996186
2,ATL,William B Hartsfield-Atlanta Intl,Atlanta,GA,USA,33.640444,-84.426944
3,AUS,Austin-Bergstrom International,Austin,TX,USA,30.194533,-97.669872
4,BDL,Bradley International,Windsor Locks,CT,USA,41.938874,-72.683228


In [15]:
airports.to_csv('../data/airports.csv', index = False)

Changed columns to be lower case. Some information will be redundant when merged with `us_pop`. 

---

## Cleaning METAR Dataset, will require most work

In [16]:
# Checking output of method before store in df
# df['id'].str.split(pat = '\t', expand = True)

# Seeing what a merge looks like with this .str method
df = df.merge(df['id'].str.split(pat = '\t', expand = True), left_index = True, right_index = True)

# renaming columns after merge
df.rename(columns = {
    0: 'time_collected',
    1: 'station'
}, inplace = True)


# rearraning columns
new_col_order = ['date', 'time_collected', 'station', 'obsv_time', 'automated', 'wind_speed', 'visibility', 'cloud_cover',]

df = df.loc[:, new_col_order].copy()

print(df.head())
print('='*40)
df.info()

         date time_collected station obsv_time automated wind_speed  \
0  2015-03-25       21:15:00    KCXP   252115Z      AUTO    08005KT   
1  2015-03-25       21:13:00    KWRB   252113Z      AUTO    06005KT   
2  2015-03-25       21:12:00    KTUL   252112Z   14006KT       10SM   
3  2015-03-25       21:11:00    KDRT   252111Z      AUTO    13007KT   
4  2015-03-25       21:10:00    KBRL   252110Z      AUTO    33009KT   

  visibility cloud_cover  
0       10SM         CLR  
1       10SM      BKN014  
2         TS      BKN035  
3       10SM      SCT023  
4        8SM      SCT016  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7731241 entries, 0 to 7731240
Data columns (total 8 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   date            object
 1   time_collected  object
 2   station         object
 3   obsv_time       object
 4   automated       object
 5   wind_speed      object
 6   visibility      object
 7   cloud_cover     object
dtypes: object(8)
m

In [17]:
# replacing the leading K in the airport codes
df['station'] = df['station'].str.replace('^K', '')

df.head()

,date,time_collected,station,obsv_time,automated,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,AUTO,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,AUTO,06005KT,10SM,BKN014
2,2015-03-25,21:12:00,TUL,252112Z,14006KT,10SM,TS,BKN035
3,2015-03-25,21:11:00,DRT,252111Z,AUTO,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,AUTO,33009KT,8SM,SCT016


---

### Notes on columns:

* `Date` and `time_collected` could be combined and turned into a datetime datatype
* `station` should have the K stripped since most airport codes are in the 3 letters (although K will sometimes be at front)
* Can probably get rid of obsv_time, but will keep for now
* need to find a way to put wind_speed in easier/layman's representation
* Fairly confident `visibility` can be represented as a string
* Possible ordinal encoder from sklearn for `cloud_cover` or make simple ranking

There is an issue with the `automated` and `wind_speed` columns. Let's see the breakdown of observations that were automated recordings and which ones were not.

In [18]:
print(f"Number of responses listed as automated: {df[df['automated'] == 'AUTO'].shape[0]}")
print(f"Number of responses listed as not automated: {df[df['automated'] != 'AUTO'].shape[0]}")
print('='*60)
print(f"Proportion of responses listed as automated: {(df[df['automated'] == 'AUTO'].shape[0])/len(df):.2f}")
print(f"Proportion of responses listed as not automated: {(df[df['automated'] != 'AUTO'].shape[0])/len(df):.2f}")

Number of responses listed as automated: 4361474
Number of responses listed as not automated: 3369767
Proportion of responses listed as automated: 0.56
Proportion of responses listed as not automated: 0.44


---

Too many observations in `df['automated']` to throw out observations that `!= 'AUTO'`

From here, `df` will be split into two separate dataframes based on the values in the `automated` column, selecting only the columns of interest to answer the problem, and then concatenated into a single dataframe.

Concatenation makes more sense than a merge because the columns will be the same in both dataframes and we aren't joining the data on a particular column or index.

In [19]:
df.head()

,date,time_collected,station,obsv_time,automated,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,AUTO,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,AUTO,06005KT,10SM,BKN014
2,2015-03-25,21:12:00,TUL,252112Z,14006KT,10SM,TS,BKN035
3,2015-03-25,21:11:00,DRT,252111Z,AUTO,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,AUTO,33009KT,8SM,SCT016


In [20]:
# Creating df_auto, only the Automated values
df_auto = df[df['automated'] == 'AUTO'].copy()

print(f'Shape of df_auto: {df_auto.shape}')
print('='*30)

df_auto.head()

Shape of df_auto: (4361474, 8)


,date,time_collected,station,obsv_time,automated,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,AUTO,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,AUTO,06005KT,10SM,BKN014
3,2015-03-25,21:11:00,DRT,252111Z,AUTO,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,AUTO,33009KT,8SM,SCT016
5,2015-03-25,21:10:00,CMX,252110Z,AUTO,26016KT,4SM,-SN


In [21]:
# Same process for non-automated values
df_non_auto = df[df['automated'] != 'AUTO'].copy()

print(f'Shape of df_auto: {df_non_auto.shape}')
print('='*30)

df_non_auto.head()

Shape of df_auto: (3369767, 8)


,date,time_collected,station,obsv_time,automated,wind_speed,visibility,cloud_cover
2,2015-03-25,21:12:00,TUL,252112Z,14006KT,10SM,TS,BKN035
7,2015-03-25,21:10:00,SZL,252110Z,03009KT,9SM,-RA,VCTS
9,2015-03-25,21:09:00,GCK,252109Z,01029G38KT,10SM,BKN018,BKN033
10,2015-03-25,21:09:00,GSP,252109Z,02009KT,7SM,FEW011,SCT021
15,2015-03-25,21:07:00,SFO,252107Z,29014KT,10SM,FEW200,21/12


In [22]:
# Only selecting columns of interest, will rename to have same columns as df_auto
df_non_auto = df_non_auto.loc[:, ['date', 'time_collected', 'station', 'automated', 'wind_speed', 'visibility', 'cloud_cover']]

print(f'Shape of df_auto: {df_non_auto.shape}')
print('='*30)

df_non_auto.head()

Shape of df_auto: (3369767, 7)


,date,time_collected,station,automated,wind_speed,visibility,cloud_cover
2,2015-03-25,21:12:00,TUL,14006KT,10SM,TS,BKN035
7,2015-03-25,21:10:00,SZL,03009KT,9SM,-RA,VCTS
9,2015-03-25,21:09:00,GCK,01029G38KT,10SM,BKN018,BKN033
10,2015-03-25,21:09:00,GSP,02009KT,7SM,FEW011,SCT021
15,2015-03-25,21:07:00,SFO,29014KT,10SM,FEW200,21/12


In [23]:
df_auto.drop(columns = 'automated', inplace = True)

df_auto.head()

,date,time_collected,station,obsv_time,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,06005KT,10SM,BKN014
3,2015-03-25,21:11:00,DRT,252111Z,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,33009KT,8SM,SCT016
5,2015-03-25,21:10:00,CMX,252110Z,26016KT,4SM,-SN


In [24]:
print(f'df_auto column names: {df_auto.columns}')
print('='*50)
print(f'df_non_auto column names: {df_non_auto.columns}')

df_auto column names: Index(['date', 'time_collected', 'station', 'obsv_time', 'wind_speed',
       'visibility', 'cloud_cover'],
      dtype='object')
df_non_auto column names: Index(['date', 'time_collected', 'station', 'automated', 'wind_speed',
       'visibility', 'cloud_cover'],
      dtype='object')


In [25]:
df_auto.head()

,date,time_collected,station,obsv_time,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,06005KT,10SM,BKN014
3,2015-03-25,21:11:00,DRT,252111Z,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,33009KT,8SM,SCT016
5,2015-03-25,21:10:00,CMX,252110Z,26016KT,4SM,-SN


In [26]:
df_non_auto.head()

,date,time_collected,station,automated,wind_speed,visibility,cloud_cover
2,2015-03-25,21:12:00,TUL,14006KT,10SM,TS,BKN035
7,2015-03-25,21:10:00,SZL,03009KT,9SM,-RA,VCTS
9,2015-03-25,21:09:00,GCK,01029G38KT,10SM,BKN018,BKN033
10,2015-03-25,21:09:00,GSP,02009KT,7SM,FEW011,SCT021
15,2015-03-25,21:07:00,SFO,29014KT,10SM,FEW200,21/12


In [27]:
df_non_auto.columns = df_auto.columns

df_non_auto.head()

,date,time_collected,station,obsv_time,wind_speed,visibility,cloud_cover
2,2015-03-25,21:12:00,TUL,14006KT,10SM,TS,BKN035
7,2015-03-25,21:10:00,SZL,03009KT,9SM,-RA,VCTS
9,2015-03-25,21:09:00,GCK,01029G38KT,10SM,BKN018,BKN033
10,2015-03-25,21:09:00,GSP,02009KT,7SM,FEW011,SCT021
15,2015-03-25,21:07:00,SFO,29014KT,10SM,FEW200,21/12


In [28]:
assert df_non_auto.shape[1] == df_auto.shape[1]

May actually make more sense to clean dataframes separately before concatenation

In [29]:
df_auto.head()

,date,time_collected,station,obsv_time,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,252115Z,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,252113Z,06005KT,10SM,BKN014
3,2015-03-25,21:11:00,DRT,252111Z,13007KT,10SM,SCT023
4,2015-03-25,21:10:00,BRL,252110Z,33009KT,8SM,SCT016
5,2015-03-25,21:10:00,CMX,252110Z,26016KT,4SM,-SN


messing with timestamps

In [33]:
df_auto['timestamp'] = pd.to_datetime(df_auto['date'] + ' ' + df_auto['time_collected'])

In [34]:
df_auto.head()

,date,time_collected,station,obsv_time,wind_speed,visibility,cloud_cover,timestamp
0,2015-03-25,21:15:00,CXP,252115Z,08005KT,10SM,CLR,2015-03-25 21:15:00
1,2015-03-25,21:13:00,WRB,252113Z,06005KT,10SM,BKN014,2015-03-25 21:13:00
3,2015-03-25,21:11:00,DRT,252111Z,13007KT,10SM,SCT023,2015-03-25 21:11:00
4,2015-03-25,21:10:00,BRL,252110Z,33009KT,8SM,SCT016,2015-03-25 21:10:00
5,2015-03-25,21:10:00,CMX,252110Z,26016KT,4SM,-SN,2015-03-25 21:10:00


In [36]:
pattern = r'G\d+KT|\d{3}|KT'

df_auto['wind_speed'].str.replace(pattern, '', regex=True)

0             05
1             05
3             07
4             09
5             16
           ...  
7731226       17
7731230    VRB04
7731231       17
7731236       07
7731239       07
Name: wind_speed, Length: 4361474, dtype: object

In [42]:
combined_df = pd.concat([df_auto, df_non_auto])

combined_df.drop(columns = 'obsv_time', inplace = True)

combined_df.reset_index(inplace = True, drop = True)

combined_df.head()

,date,time_collected,station,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,08005KT,10SM,CLR
1,2015-03-25,21:13:00,WRB,06005KT,10SM,BKN014
2,2015-03-25,21:11:00,DRT,13007KT,10SM,SCT023
3,2015-03-25,21:10:00,BRL,33009KT,8SM,SCT016
4,2015-03-25,21:10:00,CMX,26016KT,4SM,-SN


In [43]:
assert len(combined_df) == len(df), "Not the same"

---

Columns of interest are: 

- date
- time_collected
- station
- wind_speed
- visibility
- cloud cover

Need to parse out KT for wind_speed, SM for visibility, and make conditions/ordinal encoder from sklearn for cloud_cover

---

### Wind_speed:

- first three digits are true direction in degrees
- next two digits are sustained speed
- if gusts are present, next two digits are max wind speed in past few minutes

Example METAR report: METAR KJFK 291953Z 17015G25KT 10SM FEW040 BKN070 28/18 A3002 RMK AO2 SLP166 T02830183

In this example, the wind part is "17015G25KT." Here's what each section of this wind part means:

Wind Direction: "170"

The first three digits represent the wind direction in degrees true. In this case, the wind is coming from 170 degrees on the compass, which is roughly from the south-southeast direction.

Wind Speed: "15KT"

The next two digits represent the sustained wind speed in knots. In this case, the wind is blowing at a constant speed of 15 knots.
Wind Gusts: "G25KT"

We aren't too concerned with Gusts or direction of the wind. We'll merely want to parse out the windspeed, to help us get mean and medians for each station.

In [44]:
print(combined_df['wind_speed'].head())
print('='*30)
combined_df.info()

0    08005KT
1    06005KT
2    13007KT
3    33009KT
4    26016KT
Name: wind_speed, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7731241 entries, 0 to 7731240
Data columns (total 6 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   date            object
 1   time_collected  object
 2   station         object
 3   wind_speed      object
 4   visibility      object
 5   cloud_cover     object
dtypes: object(6)
memory usage: 353.9+ MB


In [39]:
# Regular expression pattern to remove wind direction, gusts, and "KT"
pattern = r'G\d+KT|\d{3}|KT'

combined_df['wind_speed'] = combined_df['wind_speed'].str.replace(pattern, '', regex = True)

In [40]:
combined_df.head()

,date,time_collected,station,wind_speed,visibility,cloud_cover
0,2015-03-25,21:15:00,CXP,05,10SM,CLR
1,2015-03-25,21:13:00,WRB,05,10SM,BKN014
3,2015-03-25,21:11:00,DRT,07,10SM,SCT023
4,2015-03-25,21:10:00,BRL,09,8SM,SCT016
5,2015-03-25,21:10:00,CMX,16,4SM,-SN


In [41]:
combined_df['wind_speed'].value_counts()

10SM         2662886
00            758930
04            402552
05            395342
03            385827
              ...   
10/M01             1
37/08              1
18/01              1
/17G25             1
R21L/P0FT          1
Name: wind_speed, Length: 898, dtype: int64

In [42]:
combined_df['wind_speed'].unique()

array(['05', '07', '09', '16', '18', '11', '08', '20', '15', '17', '12',
       '04', '03', '06', '14', '10', 'VRB02', '21', '00', 'VRB03', '22',
       '23', 'VRB05', '25', '13', '19', '24', 'VRB04', 'VRB06', '26',
       '33', '10SM', '32', '29', 'VRB01', '02', '30', '31', '01', '5SM',
       '28', '27', '34', '37', '9SM', '1', '4SM', '7SM', '3SM', '6SM',
       'M', '8SM', '19/M05', 'A0', 'CLR', 'A2', '02/M01', '3/4SM', '-RA',
       '17/02', 'A3', '11/05', 'A1', 'M09/M11', '13/06', '36', '35', '41',
       '38', '40', '43', '39', '45', '49', '15/M01', '2SM', '16/M02',
       '17/15', 'A5', '1/2SM', '22/02', '15/03', '24/14', 'A9', '30/M04',
       '28/M05', 'A4', '/07', 'R35/0VP0FT', 'R35/0V0FT', 'A6', 'A8',
       'R35/0FT', '07/M02', 'A7', 'M01/M05', 'BKN', '26/18', '2', '1SM',
       '19/20', 'OVC', 'SCT', '09/07', 'VCTS', '18/01', 'FEW', '44',
       '00/M02', '-SN', '14/05', '01/', 'TS', '21/19', '12/10', '10/11',
       '7/08', '12/M08', 'VRB/04', '21/', '18/M02', 'M01/M03', 

In [61]:
combined_df['wind_speed'].head(1).apply(lambda x: int(x))

0    5
Name: wind_speed, dtype: int64

The values that couldn't be converted to numeric with the RegEx will be replaced with nan's.

Eventually we'll try various imputation techniques to clean the data.

* Simple Imputer (mean, median)
* kNN Imputer

In [45]:
wind_speed = []
for i in combined_df['wind_speed']:
    try:
        wind_speed.append(int(i))
    except:
        wind_speed.append(np.nan)

In [46]:
assert len(wind_speed) == len(combined_df['wind_speed'])

In [55]:
pd.DataFrame(wind_speed).value_counts(ascending = False)

0.0     768193
4.0     402563
5.0     395372
3.0     385856
6.0     360913
7.0     339065
8.0     286981
9.0     261502
10.0    201525
11.0    167766
12.0    133469
13.0    109099
14.0     86660
15.0     67498
2.0      66943
16.0     54096
1.0      51140
17.0     40948
18.0     32641
19.0     24889
20.0     19362
21.0     14560
22.0     11310
23.0      8383
24.0      6317
25.0      4754
26.0      3570
27.0      2740
28.0      1948
29.0      1524
30.0      1107
31.0       885
32.0       643
33.0       495
34.0       367
35.0       264
36.0       171
37.0       128
38.0        84
39.0        72
40.0        59
41.0        47
43.0        23
42.0        19
44.0        11
45.0         8
47.0         5
49.0         3
46.0         3
99.0         2
48.0         1
50.0         1
51.0         1
52.0         1
57.0         1
67.0         1
dtype: int64

In [54]:
pd.__version__

'1.1.3'

---


### Cloud_cover:

- CLEAR = sky clear
- FEW = 0-2 eighths
- SCT = scattered 3-4 eighths
- BKN = Broken - 5-7 eighths
- OVC = Overcast - 8 eighths

cloud cover, thinking rules based approach eg:

- clear = 5
- few = 4
- sct = 3
- bkn = 2
- else: 1

